In [1]:
import numpy as np
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, accuracy_score
import xgboost as xgb
from hyperopt import STATUS_OK, hp, fmin, tpe
from tqdm import tqdm

import shap
import numpy as np, os, sys

In [ ]:
df = pd.read_csv("../real_time_sepsis_development/real_time_data/2021_6hr_preprocessed_48_0426.csv")
df["SepsisLabel"] = df["SepsisLabel"].astype(int)

In [2]:
id_sepsis = np.load("../real_time_sepsis_development/real_time_data/sepsis_48.npy")
id_nosepsis = np.load("../real_time_sepsis_development/real_time_data/control_resampled_48.npy")

print("Number of sepsis patients: {}".format(len(id_sepsis)))
print("Number of non-sepsis patients: {}".format(len(id_nosepsis)))

Number of sepsis patients: 2791
Number of non-sepsis patients: 27910


# make sure to create "xgb_model" folder 

## Training

In [21]:
def downsample(x):
    
    pos = x[x["SepsisLabel"] == 1]
    neg = x[x["SepsisLabel"] == 0]
    
    if len(pos) < len(neg):
        neg = neg.sample(n=len(pos), replace = False, random_state = 10002)
        
    new = pos.append(neg)
    new = new.sample(frac = 1, replace = False)
    
    return new

In [22]:
def BO_TPE(X_train, y_train, X_val, y_val):
    "Hyperparameter optimization"
    train = xgb.DMatrix(X_train, label=y_train)
    val = xgb.DMatrix(X_val, label=y_val)
    X_val_D = xgb.DMatrix(X_val)

    def objective(params):
        xgb_model = xgb.train(params, dtrain=train, num_boost_round=1000, evals=[(val, 'eval')],
                              verbose_eval=False, early_stopping_rounds=80)
        y_vd_pred = xgb_model.predict(X_val_D, ntree_limit=xgb_model.best_ntree_limit)
        y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]

        acc = accuracy_score(y_val, y_val_class)
        loss = 1 - acc

        return {'loss': loss, 'params': params, 'status': STATUS_OK}

    max_depths = [3, 4]
    learning_rates = [0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.15, 0.2]
    subsamples = [0.5, 0.6, 0.7, 0.8, 0.9]
    colsample_bytrees = [0.5, 0.6, 0.7, 0.8, 0.9]
    reg_alphas = [0.0, 0.005, 0.01, 0.05, 0.1]
    reg_lambdas = [0.8, 1, 1.5, 2, 4]

    space = {
        'max_depth': hp.choice('max_depth', max_depths),
        'learning_rate': hp.choice('learning_rate', learning_rates),
        'subsample': hp.choice('subsample', subsamples),
        'colsample_bytree': hp.choice('colsample_bytree', colsample_bytrees),
        'reg_alpha': hp.choice('reg_alpha', reg_alphas),
        'reg_lambda': hp.choice('reg_lambda', reg_lambdas),
    }

    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20)

    best_param = {'max_depth': max_depths[(best['max_depth'])],
                  'learning_rate': learning_rates[(best['learning_rate'])],
                  'subsample': subsamples[(best['subsample'])],
                  'colsample_bytree': colsample_bytrees[(best['colsample_bytree'])],
                  'reg_alpha': reg_alphas[(best['reg_alpha'])],
                  'reg_lambda': reg_lambdas[(best['reg_lambda'])]
                  }

    return best_param

def train_model(k, X_train, y_train, X_val, y_val, save_model_dir):
  
    print('*************************************************************')
    print('{}th training ..............'.format(k + 1))
    print('Hyperparameters optimization')
    best_param = BO_TPE(X_train, y_train, X_val, y_val)
    print("obtained best_param")
    xgb_model = xgb.XGBClassifier(max_depth = best_param['max_depth'],
                                  eta = best_param['learning_rate'],
                                  n_estimators = 1000,
                                  subsample = best_param['subsample'],
                                  colsample_bytree = best_param['colsample_bytree'],
                                  reg_alpha = best_param['reg_alpha'],
                                  reg_lambda = best_param['reg_lambda'],
                                  objective = "binary:logistic"
                                  )

    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='error',
                  early_stopping_rounds=80, verbose=False)

    y_tr_pred = (xgb_model.predict_proba(X_train, ntree_limit=xgb_model.best_ntree_limit))[:, 1]
    
    train_auc = roc_auc_score(y_train, y_tr_pred)
    print('training dataset AUC: ' + str(train_auc))
    y_tr_class = [0 if i <= 0.5 else 1 for i in y_tr_pred]
    acc = accuracy_score(y_train, y_tr_class)
    print('training dataset acc: ' + str(acc))

    y_vd_pred = (xgb_model.predict_proba(X_val, ntree_limit=xgb_model.best_ntree_limit))[:, 1]

    valid_auc = roc_auc_score(y_val, y_vd_pred)
    print('validation dataset AUC: ' + str(valid_auc))
    y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]
    acc = accuracy_score(y_val, y_val_class)
    print('validation dataset acc: ' + str(acc))
    print('************************************************************')
    # save the model
    
    np.save("y_train" + str(k)+".npy", y_train)
    np.save("y_train_pred" + str(k)+".npy", y_tr_pred)
    np.save("y_val" + str(k)+".npy", y_val)
    np.save("y_val_pred" + str(k)+".npy", y_vd_pred)
    
    save_model_path = save_model_dir + 'model{}.mdl'.format(k + 1)
    xgb_model.get_booster().save_model(fname=save_model_path)


In [23]:
train_sets = []
val_sets = []
ks = []
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)) in zip(enumerate(kfold.split(train_nosepsis)), enumerate(kfold.split(train_sepsis))):
    train_sets.append(np.append(train_nosepsis[train0_index], train_sepsis[train1_index]))
    val_sets.append(np.append(train_nosepsis[val0_index], train_sepsis[val1_index]))
    ks.append(k)

In [11]:
np.save("trainsets_ids.npy", train_sets)
np.save("valsets_ids.npy", val_sets)
np.save("ks.npy", ks)

In [35]:
for k in ks:
    print(k)
    train_set = train_sets[k]
    case = df[df["csn"].isin(train_set)].reset_index(drop= True)
    #case = case.groupby(by = ["Unit1", "Unit2"]).apply(lambda x: downsample(x)).reset_index(drop = True)
    case = downsample(case)
    
    x_train = case.drop(["csn", "pat_id", "LOS", "rel_time", "SepsisLabel"], axis = 1).values
    y_train = case["SepsisLabel"].values

    print(np.shape(x_train))
    print(np.shape(y_train))
    print(sum(y_train))
    
    
    val_set = val_sets[k]
    case = df[df["csn"].isin(val_set)].reset_index(drop= True)
    #case = case.groupby(by = ["Unit1", "Unit2"]).apply(lambda x: downsample(x)).reset_index(drop = True)
    case = downsample(case)
    
    x_val = case.drop(["csn", "pat_id", "LOS", "rel_time", "SepsisLabel"], axis = 1).values
    y_val = case["SepsisLabel"].values

    train_model(k, x_train, y_train, x_val, y_val, save_model_dir = './xgb_model/')

    

0
(37582, 167)
(37582,)
18791
*************************************************************
1th training ..............
Hyperparameters optimization
100%|██████████| 20/20 [41:29<00:00, 124.49s/trial, best loss: 0.2105724939679754] 
obtained best_param
training dataset AUC: 0.9188443045106887
training dataset acc: 0.8350540152200522
validation dataset AUC: 0.8597879109899351
validation dataset acc: 0.7849309059004168
************************************************************
1
(36864, 167)
(36864,)
18432
*************************************************************
2th training ..............
Hyperparameters optimization
100%|██████████| 20/20 [40:18<00:00, 120.93s/trial, best loss: 0.24491663277755182]
obtained best_param
training dataset AUC: 0.8980609620058977
training dataset acc: 0.8106011284722222
validation dataset AUC: 0.8456392234020614
validation dataset acc: 0.7544733631557544
************************************************************
2
(37848, 167)
(37848,)
18924
******

# TESTING

In [36]:
def load_model_predict(X_test, k_fold, path):
    "ensemble the five XGBoost models by averaging their output probabilities"
    test_pred = np.zeros((X_test.shape[0], k_fold))
    X_test = xgb.DMatrix(X_test)
    for k in range(k_fold):
        model_path_name = path + 'model{}.mdl'.format(k+1)
        xgb_model = xgb.Booster(model_file = model_path_name)
        y_test_pred = xgb_model.predict(X_test)
        test_pred[:, k] = y_test_pred
    test_pred = pd.DataFrame(test_pred)
    result_pro = test_pred.mean(axis=1)

    return result_pro


def predict(data_set,
            data_dir,
            model_path,
            risk_threshold
            ):
   
    result = pd.DataFrame()
    print(len(data_set))
    count = 0
    
    for p in data_set:
        count+=1
        print(count)
        
        patient_df = data_dir[data_dir["csn"] == p]
        
        features = patient_df.drop(["csn", "pat_id", "LOS", "rel_time", "SepsisLabel"], axis = 1).values
        
        labels = patient_df["SepsisLabel"].values
        
        predict_pro = load_model_predict(features, k_fold = 5, path = './' + model_path + '/')
        
        PredictedProbability = np.array(predict_pro)
        PredictedLabel = [0 if i <= risk_threshold else 1 for i in predict_pro]
        
        temp_result = patient_df.copy()
        temp_result = temp_result[["csn", "pat_id", "LOS", "rel_time", "SepsisLabel"]]
        temp_result["PredictedProbability"] = PredictedProbability
        temp_result["PredictedSepsisLabel"] = PredictedLabel
        result = result.append(temp_result)
    
    return result
        
        

In [ ]:
test_set = np.load('./real_time_data/test_set.npy')
test_data_path = df[df["csn"].isin(test_set)]
model_path = "xgb_model"

result = predict(test_set, test_data_path, model_path, 0.48)
result.to_csv("./xgb_model/prediction_results_0426.csv", index = False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



4606
1
